In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/pSp_self_customize')


Mounted at /content/drive


In [2]:
! pip install --upgrade azure-cognitiveservices-vision-face


     |████████████████████████████████| 71kB 3.7MB/s 
     |████████████████████████████████| 92kB 5.0MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 


In [3]:
import glob

import numpy as np

from PIL import Image, ImageDraw

from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person



In [4]:
KEY = "ac5011e2955b41ef8f0a3f26b664bfc6"

ENDPOINT = "https://zy.cognitiveservices.azure.com/"

# Create an authenticated FaceClient.
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))


In [5]:
source_path = "./image_datasets/ffhq0k_5k"
target_path = "./image_datasets/inference_results"


In [6]:
source_image_names = sorted(glob.glob(os.path.join(source_path, "*.jpg")) + glob.glob(os.path.join(source_path, "*.png")))
target_image_names = sorted(glob.glob(os.path.join(target_path, "*.jpg")) + glob.glob(os.path.join(target_path, "*.png")))

print(source_image_names)
print(target_image_names)

for source_img, target_img in zip(source_image_names[:5], target_image_names[:5]):
  print("~~~~~~~~~~~~~~~~~~~~~~~~")
  print(os.path.basename(source_img))
  print(os.path.basename(target_img))


['./image_datasets/ffhq0k_5k/00000.png', './image_datasets/ffhq0k_5k/00001.png', './image_datasets/ffhq0k_5k/00002.png', './image_datasets/ffhq0k_5k/00003.png', './image_datasets/ffhq0k_5k/00004.png', './image_datasets/ffhq0k_5k/00005.png', './image_datasets/ffhq0k_5k/00006.png', './image_datasets/ffhq0k_5k/00007.png', './image_datasets/ffhq0k_5k/00008.png', './image_datasets/ffhq0k_5k/00009.png', './image_datasets/ffhq0k_5k/00010.png', './image_datasets/ffhq0k_5k/00011.png', './image_datasets/ffhq0k_5k/00012.png', './image_datasets/ffhq0k_5k/00013.png', './image_datasets/ffhq0k_5k/00014.png', './image_datasets/ffhq0k_5k/00015.png', './image_datasets/ffhq0k_5k/00016.png', './image_datasets/ffhq0k_5k/00017.png', './image_datasets/ffhq0k_5k/00018.png', './image_datasets/ffhq0k_5k/00019.png', './image_datasets/ffhq0k_5k/00020.png', './image_datasets/ffhq0k_5k/00021.png', './image_datasets/ffhq0k_5k/00022.png', './image_datasets/ffhq0k_5k/00023.png', './image_datasets/ffhq0k_5k/00024.png',

In [7]:
# Exploring how to load local images
test_image_path = './image_datasets/inversion_images/donal_trump.jpg'

test_image = open(test_image_path,'rb')
print(type(test_image))

detected_test_face = face_client.face.detect_with_stream(test_image, detection_model='detection_02')
test_image_id = detected_test_face[0].face_id
print('{} face detected from test image {}.'.format(len(detected_test_face), test_image_id))


<class '_io.BufferedReader'>
1 face detected from test image 02af6c39-6c10-42bf-bf61-becfdc0f5924.


In [8]:
verify_confidence = []
for source_name, target_name in zip(source_image_names, target_image_names):
  source_img, target_img = open(source_name,'rb'), open(target_name,'rb')
  
  detected_source_face = face_client.face.detect_with_stream(source_img, detection_model='detection_02')
  detected_target_face = face_client.face.detect_with_stream(target_img, detection_model='detection_02')

  source_id = detected_source_face[0].face_id
  target_id = detected_target_face[0].face_id
  
  verify_result = face_client.face.verify_face_to_face(source_id, target_id)
  verify_confidence.append(verify_result.confidence)
  
  print('Faces from {} & {} are of the same person, with confidence: {}'
        .format(os.path.basename(source_name), os.path.basename(target_name), verify_result.confidence)
        if verify_result.is_identical
        else 'Faces from {} & {} are of a different person, with confidence: {}'
        .format(os.path.basename(source_name), os.path.basename(target_name), verify_result.confidence))
  


Faces from 00000.png & 00000.png are of the same person, with confidence: 0.83295
Faces from 00001.png & 00001.png are of a different person, with confidence: 0.49606
Faces from 00002.png & 00002.png are of the same person, with confidence: 0.59595
Faces from 00003.png & 00003.png are of the same person, with confidence: 0.84124
Faces from 00004.png & 00004.png are of the same person, with confidence: 0.61514
Faces from 00005.png & 00005.png are of the same person, with confidence: 0.51445


APIErrorException: ignored

In [ ]:
avg_verify_confidence = np.mean(verify_confidence)
print("Average Verify Confidence is: {}".format(avg_verify_confidence))
